# Here is some approaches that will be used while matchmaking

- Some of them better for large amount of users comparing in performance (Clustering + SA)
- Some will be better for average amount of users (Clustering + Greedy, GA)
- Other better for small (greedy)

In [30]:
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Dict, Tuple, Optional, DefaultDict
import random
from datetime import datetime, timedelta
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.cluster import KMeans
from math import exp
from copy import deepcopy
from collections import defaultdict
from sklearn.metrics import pairwise_distances
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /Users/rail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/rail/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/rail/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Format

#### Users_data (users that will be used in matching)
```json
{
    "users_data": [
        {
            "user_id": "...",
            "bio": "...",
            "tags": ["...", "..."]
        }
        // more users
    ]
}
```
#### History
```json
{
    "history": [
        {//one session
            "user_id1": "...",
            "user_id2": "...",
            "timestamp": "..."
        }
        // other
    ]
}

## Get info

In [31]:
def get_bio(df):
   pass 

def get_history(df):
   pass

## Preprocess

In [32]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text: str, stem_words: bool = True, remove_stopwords: bool = True) -> str:
    
    text = text.lower()
    text = re.sub(r'\n+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'\d+', '', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    
    words = word_tokenize(text)
    
    if remove_stopwords:
        words = [word for word in words if word not in stop_words]
    
    if stem_words:
        words = [stemmer.stem(word) for word in words]
    
    return ' '.join(words)

## Test data

In [33]:
users_data = [
    {
        "user_id": str(i),
        "bio": random.choice([
            f"Software engineer passionate about {random.choice(['AI', 'web development', 'cloud computing'])}",
            f"Data scientist specializing in {random.choice(['NLP', 'computer vision', 'time series'])}",
            f"UX designer with focus on {random.choice(['mobile apps', 'accessibility', 'user research'])}",
            f"Product manager interested in {random.choice(['blockchain', 'edtech', 'healthtech'])}",
            "Digital marketer growth hacking enthusiast",
            "DevOps engineer building scalable infrastructure",
            "Cybersecurity expert focusing on penetration testing"
        ]),
        "tags": random.sample([
            "tech", "AI", "programming", "data", "design", 
            "startups", "entrepreneurship", "marketing",
            "crypto", "health", "education", "finance",
            "music", "art", "travel", "sports", "reading"
        ], k=random.randint(2, 5))
    } 
    for i in range(1, 32)
]


history = []
user_ids = [str(i) for i in range(1, 32)]
date = datetime.now() - timedelta(days=180) 

for _ in range(20):
    user1, user2 = random.sample(user_ids, 2)
    history.append({
        "user_id1": user1,
        "user_id2": user2,
        "timestamp": date.strftime("%Y-%m-%d")
    })
    date += timedelta(days=random.randint(7, 21))
    
    user_ids.remove(user1)
    if user2 in user_ids:
        user_ids.remove(user2)
    if len(user_ids) < 2:
        user_ids = [str(i) for i in range(1, 32)] 

for user in users_data:
    user["bio"] = preprocess_text(user["bio"])

## Tag distances
Must return value such that: 
- 0 are more similar, ..., 1 are not similar

In [34]:
def jaccard_distance(tags1: List[str], tags2: List[str]) -> float:
    set1, set2 = set(tags1), set(tags2)
    
    intersection = len(set1 & set2)
    union = len(set1 | set2)

    return 1 - intersection / union if union != 0 else 1.0

def semantic_tag_distance(tags1: List[str], tags2: List[str], embedder: SentenceTransformer) -> float:
    text1 = " ".join(tags1)
    text2 = " ".join(tags2)

    emb1 = embedder.encode(text1)
    emb2 = embedder.encode(text2)
    if hasattr(emb1, "cpu"):
        emb1 = emb1.cpu().numpy()
    if hasattr(emb2, "cpu"):
        emb2 = emb2.cpu().numpy()
    
    return 1 - cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]

## Genetics approach

In [45]:
class GeneticMatcher:
    def __init__(
        self,
        tag_distance_func: str = "jaccard",  # "jaccard" or "semantic"
        coef_a: float = 0.6,
        coef_b: float = 0.4,
        penalty_multiplier: float = 0.9,
        embedding_model: str = "all-MiniLM-L6-v2",
        pop_size: int = 50,
        generations: int = 100,
        mutation_rate: float = 0.1
    ):
        self.coef_a = coef_a
        self.coef_b = coef_b
        self.penalty = penalty_multiplier
        self.pop_size = pop_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.embedder = SentenceTransformer(embedding_model)
        self.tag_distance = {
            "jaccard": jaccard_distance,
            "semantic": lambda t1, t2: semantic_tag_distance(t1, t2, self.embedder)
        }.get(tag_distance_func, jaccard_distance)
        

    def compute_pair_score(self, user1: Dict, user2: Dict, history: List[Dict]) -> float:
        bio_sim = cosine_similarity([user1["bio_embedding"]], [user2["bio_embedding"]])[0][0]
        
        tag_dist = self.tag_distance(user1["tags"], user2["tags"])
        
        base_score = self.coef_a * bio_sim + self.coef_b * (1 - tag_dist)
        
        weeks_passed = self.get_weeks_since_last_meeting(user1["user_id"], user2["user_id"],history)
        penalty = self.penalty ** weeks_passed if weeks_passed != float("inf") else 0

        return base_score * (1 - penalty)
    
    def get_weeks_since_last_meeting(self, user_id1: str, user_id2: str, history: List[Dict]) -> float:
        for event in history:
            if {event["user_id1"], event["user_id2"]} == {user_id1, user_id2}:
                last_meeting = datetime.strptime(event["timestamp"], "%Y-%m-%d")
                return (datetime.now() - last_meeting).days // 7
            
        return float("inf")
    
    def create_individual(self, users: List[Dict]) -> List[Tuple[int, int]]:
        indices = list(range(len(users)))
        random.shuffle(indices)
        
        pairs = [(indices[i], indices[i+1]) for i in range(0, len(indices)-1, 2)]
        
        if len(indices) % 2 != 0:
            pairs.append((indices[-1], -1))
            
        return pairs
    
    def fitness(self, individual: List[Tuple[int, int]], users: List[Dict], history: List[Dict]) -> float:
        total_score = 0.0
        valid_pairs = 0
        count_non_valid = 0
        for i, j in individual:
            if j == -1:
                if count_non_valid == 0:
                    count_non_valid += 1
                    continue
                else:
                    score -= 1
                    valid_pairs += 1
                    continue
            score = self.compute_pair_score(users[i], users[j], history)
            total_score += score
            valid_pairs += 1

        return total_score / valid_pairs if valid_pairs > 0 else 0.0
    
    def crossover(self, parent1: List[Tuple[int, int]], parent2: List[Tuple[int, int]], users: List[Dict]) -> List[Tuple[int, int]]:
        crossover_point = random.randint(1, len(parent1)-1)
        child = parent1[:crossover_point] + parent2[crossover_point:]

        return self.repair_individual(child, users)
    
    def mutate(self, individual: List[Tuple[int, int]], users: List[Dict]) -> List[Tuple[int, int]]:

        if random.random() < self.mutation_rate:
            idx1, idx2 = random.sample(range(len(individual)), 2)
            i1, j1 = individual[idx1]
            i2, j2 = individual[idx2]
            individual[idx1] = (i1, j2)
            individual[idx2] = (i2, j1)

        return self.repair_individual(individual, users)
    
    
    def repair_individual(self, individual: List[Tuple[int, int]], users: List[Dict]) -> List[Tuple[int, int]]:
        used = set()
        new_individual = []
        unpaired = None
        
        for i, j in individual:
            if i not in used and (j == -1 or j not in used):
                if j == -1:
                    if unpaired is None:
                        unpaired = i
                        used.add(i)
                        new_individual.append((i, -1))
                else:
                    new_individual.append((i, j))
                    used.add(i)
                    used.add(j)
        
        all_users = set(range(len(users)))
        leftover = list(all_users - used)
        
        for k in range(0, len(leftover)-1, 2):
            new_individual.append((leftover[k], leftover[k+1]))
        
        if len(leftover) % 2 != 0:
            if unpaired is None:
                new_individual.append((leftover[-1], -1))
        
        return new_individual

    

    def match(self, users_data: List[Dict], history: List[Dict] = None) -> List[Tuple[str, Optional[str]]]:
        if history is None:
            history = []
        
        for user in users_data:
            user["bio_embedding"] = self.embedder.encode(user["bio"])
            if hasattr(user["bio_embedding"], "cpu"):
                user["bio_embedding"] = user["bio_embedding"].cpu().numpy()
        
        population = [self.create_individual(users_data) for _ in range(self.pop_size)]
        
        for generation in range(self.generations):
            print(f"Generation {generation+1}/{self.generations}")
            fitness_scores = [self.fitness(ind, users_data, history) for ind in population]
            print(f"Best fitness in generation {generation+1}: {max(fitness_scores)}")
            
            selected = []
            for _ in range(self.pop_size):
                candidates = random.sample(list(zip(population, fitness_scores)), 3)
                best = max(candidates, key=lambda x: x[1])[0]
                selected.append(best)
            
            
            new_population = []
            for i in range(0, self.pop_size, 2):
                parent1, parent2 = selected[i], selected[i+1]
                child1 = self.crossover(parent1, parent2, users_data)
                child2 = self.crossover(parent2, parent1, users_data)
                new_population.extend([
                    self.mutate(child1, users_data),
                    self.mutate(child2, users_data)
                ])
            
            population = new_population
        
        best_individual = max(population, key=lambda ind: self.fitness(ind, users_data, history))
        
        result = []
        for i, j in best_individual:
            user1 = users_data[i]["user_id"]
            user2 = users_data[j]["user_id"] if j != -1 else None
            result.append((user1, user2))

        return result
                

In [46]:
matcher = GeneticMatcher(
    tag_distance_func="semantic",
    coef_a=0.6,
    coef_b=0.4,
    penalty_multiplier=0.9,
    embedding_model="all-MiniLM-L6-v2",
    pop_size=6,
    generations=100,
    mutation_rate=0.4
)

matcher.match(users_data, history)

Generation 1/100
Best fitness in generation 1: 0.32473519444465637
Generation 2/100
Best fitness in generation 2: 0.34599342942237854
Generation 3/100
Best fitness in generation 3: 0.34599342942237854
Generation 4/100
Best fitness in generation 4: 0.34599342942237854
Generation 5/100
Best fitness in generation 5: 0.34599342942237854
Generation 6/100
Best fitness in generation 6: 0.3543155789375305
Generation 7/100
Best fitness in generation 7: 0.35586783289909363
Generation 8/100
Best fitness in generation 8: 0.35453101992607117
Generation 9/100
Best fitness in generation 9: 0.3737639784812927
Generation 10/100
Best fitness in generation 10: 0.376390278339386
Generation 11/100
Best fitness in generation 11: 0.3955304026603699
Generation 12/100
Best fitness in generation 12: 0.3955304026603699
Generation 13/100
Best fitness in generation 13: 0.3955304026603699
Generation 14/100
Best fitness in generation 14: 0.3955304026603699
Generation 15/100
Best fitness in generation 15: 0.395530402

[('1', '18'),
 ('12', '13'),
 ('14', '19'),
 ('16', None),
 ('11', '31'),
 ('23', '29'),
 ('21', '20'),
 ('3', '25'),
 ('15', '30'),
 ('7', '4'),
 ('9', '6'),
 ('10', '26'),
 ('28', '22'),
 ('17', '2'),
 ('27', '8'),
 ('5', '24')]

## Greedy approach

In [37]:
class GreedyMatcher:
    def __init__(
        self,
        coef_a: float = 0.6,
        coef_b: float = 0.4,
        penalty_multiplier: float = 0.85,
        tag_distance_func: str = "jaccard",
        embedding_model: str = "all-MiniLM-L6-v2"
    ):
        self.coef_a = coef_a
        self.coef_b = coef_b
        self.penalty = penalty_multiplier
        self.tag_distance = {
            "jaccard": jaccard_distance,
            "semantic": lambda t1, t2: semantic_tag_distance(t1, t2, self.embedder)
        }.get(tag_distance_func, jaccard_distance)

        self.embedder = SentenceTransformer(embedding_model)

    def get_weeks_since_last_meeting(self, user_id1: str, user_id2: str, history: List[Dict]) -> float:
        for event in history:
            if {event["user_id1"], event["user_id2"]} == {user_id1, user_id2}:
                last_meeting = datetime.strptime(event["timestamp"], "%Y-%m-%d")
                return (datetime.now() - last_meeting).days / 7
        return float("inf") 

    def compute_pair_score(self, user1: Dict, user2: Dict, history: List[Dict]) -> float:
        bio_sim = cosine_similarity([user1["bio_embedding"]], [user2["bio_embedding"]])[0][0]
        
        tag_dist = self.tag_distance(user1["tags"], user2["tags"])
        
        base_score = self.coef_a * bio_sim + self.coef_b * (1 - tag_dist)
        
        weeks_passed = self.get_weeks_since_last_meeting(user1["user_id"], user2["user_id"],history)
        penalty = self.penalty ** weeks_passed if weeks_passed != float("inf") else 0

        return base_score * (1 - penalty)

    def match(self, users_data: List[Dict], history: List[Dict] = None) -> List[Tuple[str, Optional[str]]]:
        if history is None:
            history = []
        
        for user in users_data:
            embedding = self.embedder.encode(user["bio"])
            if hasattr(embedding, "cpu"):
                embedding = embedding.cpu().numpy()
            user["bio_embedding"] = embedding
        
        # list of all pairs of users with their scores
        pairs = []
        n = len(users_data)
        for i in range(n):
            for j in range(i + 1, n):
                user1 = users_data[i]
                user2 = users_data[j]
                score = self.compute_pair_score(user1, user2, history)
                pairs.append((score, i, j))
        
        pairs.sort(reverse=True, key=lambda x: x[0])

        # greedy matching
        matched_pairs = []
        total_score = 0.0
        pair_count = 0
        used_indices = set()
        
        for score, i, j in pairs:
            if i not in used_indices and j not in used_indices:
                matched_pairs.append((users_data[i]["user_id"], users_data[j]["user_id"]))
                total_score += score
                pair_count += 1
                used_indices.update([i, j])
                if len(used_indices) == n:
                    break
        
        all_indices = set(range(n))
        leftover_indices = all_indices - used_indices
        for idx in leftover_indices:
            matched_pairs.append((users_data[idx]["user_id"], None)) # "Sori brat" case
        
        print(f"Average score: {total_score / pair_count if pair_count > 0 else 0.0}")
        return matched_pairs

In [38]:
matcher = GreedyMatcher(
    coef_a=0.7, 
    coef_b=0.3,
    penalty_multiplier=0.9,
    tag_distance_func="jaccard"
)
matcher.match(users_data, history)


Average score: 0.6923855543136597


[('12', '13'),
 ('2', '26'),
 ('7', '11'),
 ('20', '24'),
 ('1', '14'),
 ('4', '23'),
 ('15', '22'),
 ('19', '25'),
 ('17', '18'),
 ('6', '27'),
 ('5', '29'),
 ('8', '9'),
 ('21', '31'),
 ('3', '10'),
 ('16', '28'),
 ('30', None)]

## Clustering + greedy

In [39]:
class BalancedClusterMatcher:
    def __init__(
        self,
        n_clusters: int = 5,
        coef_a: float = 0.6,
        coef_b: float = 0.4,
        penalty_multiplier: float = 0.85,
        embedding_model: str = "all-MiniLM-L6-v2",
        tag_distance_func: str = "jaccard"
    ):
        self.n_clusters = n_clusters
        self.coef_a = coef_a
        self.coef_b = coef_b
        self.penalty = penalty_multiplier
        self.embedder = SentenceTransformer(embedding_model)
        self.tag_distance = {
            "jaccard": jaccard_distance,
            "semantic": lambda t1, t2: semantic_tag_distance(t1, t2, self.embedder)
        }.get(tag_distance_func, jaccard_distance)

    def match(self, users_data: List[Dict], history: Optional[List[Dict]] = None) -> Tuple[List[Tuple[str, Optional[str], float]], float]:
        if history is None:
            history = []

        embeddings: List = []
        for user in users_data:
            embedding = self.embedder.encode(user["bio"])
            if hasattr(embedding, "cpu"):
                embedding = embedding.cpu().numpy()
            user["bio_embedding"] = embedding
            embeddings.append(embedding)
        kmeans = KMeans(n_clusters=min(self.n_clusters, len(users_data)//2), init='k-means++', random_state=42)
        clusters = kmeans.fit_predict(embeddings)

        cluster_users: DefaultDict[int, List[Dict]] = defaultdict(list)
        for idx, cluster_id in enumerate(clusters):
            cluster_users[cluster_id].append(users_data[idx])

        main_pairs: List[Tuple[Dict, Dict, float]] = []
        leftover_users: List[Dict] = []

        for cluster_id, users in cluster_users.items():
            if len(users) % 2 == 0:
                cluster_pairs = self.greedy_match_in_cluster(users, history)
                main_pairs.extend(cluster_pairs)
            else:
                cluster_pairs = self.greedy_match_in_cluster(users[:-1], history)
                main_pairs.extend(cluster_pairs)
                leftover_users.append(users[-1])

        if len(leftover_users) >= 2:
            leftover_pairs = self.greedy_match_in_cluster(leftover_users, history)
            main_pairs.extend(leftover_pairs)
            leftover_users = []

        final_pairs: List[Tuple[str, Optional[str], float]] = []
        total_score = 0.0
        pair_count = 0

        for pair in main_pairs:
            final_pairs.append((pair[0]["user_id"], pair[1]["user_id"], pair[2]))
            total_score += pair[2]
            pair_count += 1

        for user in leftover_users:
            final_pairs.append((user["user_id"], None, 0.0))

        avg_score = total_score / pair_count if pair_count > 0 else 0.0

        return final_pairs, avg_score

    def greedy_match_in_cluster(self, users: List[Dict], history: List[Dict]) -> List[Tuple[Dict, Dict, float]]:
        n = len(users)
        if n < 2:
            return []
        pairs = []
        for i in range(n):
            for j in range(i + 1, n):
                score = self.compute_pair_score(users[i], users[j], history)
                pairs.append((score, i, j))

        pairs.sort(reverse=True, key=lambda x: x[0])
        used = set()
        result = []
        for score, i, j in pairs:
            if i not in used and j not in used:
                result.append((users[i], users[j], score))
                used.add(i)
                used.add(j)
        return result

    def compute_pair_score(self, user1: Dict, user2: Dict, history: List[Dict]) -> float:
        bio_sim = cosine_similarity([user1["bio_embedding"]], [user2["bio_embedding"]])[0][0]
        tag_dist = self.tag_distance(user1["tags"], user2["tags"])
        base_score = self.coef_a * bio_sim + self.coef_b * (1 - tag_dist)
        weeks_passed = self.get_weeks_since_last_meeting(user1["user_id"], user2["user_id"], history)
        penalty = self.penalty ** weeks_passed if weeks_passed != float("inf") else 0
        return base_score * (1 - penalty)

    def get_weeks_since_last_meeting(self, user_id1: str, user_id2: str, history: List[Dict]) -> float:
        for event in history:
            if {event["user_id1"], event["user_id2"]} == {user_id1, user_id2}:
                last_meeting = datetime.strptime(event["timestamp"], "%Y-%m-%d")
                return (datetime.now() - last_meeting).days / 7
        return float("inf")

In [40]:
matcher = BalancedClusterMatcher(
    n_clusters=5,
    coef_a=0.7, 
    coef_b=0.3,
    penalty_multiplier=0.9
)
result, avg_score = matcher.match(users_data, history)
print(f"Average score: {avg_score}")
for user1, user2, score in result:
    print(f"({user1}, {user2})")

Average score: 0.6501028537750244
(20, 24)
(1, 14)
(5, 29)
(21, 31)
(2, 26)
(19, 25)
(17, 18)
(3, 10)
(7, 11)
(15, 22)
(4, 16)
(6, 27)
(8, 9)
(12, 13)
(28, 30)
(23, None)


/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


## Clustering + Simulated Anhealing

In [41]:
class OptimizedClusterAnnealingMatcher:
    def __init__(self,
                 n_clusters: int = 5,
                 coef_a: float = 0.6,
                 coef_b: float = 0.4,
                 initial_temp: float = 1000,
                 cooling_rate: float = 0.95,
                 n_iterations: int = 500,
                 penalty_multiplier: float = 0.85,
                 embedding_model: str = "all-MiniLM-L6-v2",
                 tag_distance_func: str = "jaccard"):
        
        self.n_clusters = n_clusters
        self.coef_a = coef_a
        self.coef_b = coef_b
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.n_iterations = n_iterations
        self.penalty = penalty_multiplier
        self.embedder = SentenceTransformer(embedding_model)
        self.tag_distance = {
            "jaccard": jaccard_distance,
            "semantic": lambda t1, t2: semantic_tag_distance(t1, t2, self.embedder)
        }.get(tag_distance_func, jaccard_distance)

    def cluster_users(self, users_data: List[Dict]) -> Tuple[np.ndarray, np.ndarray]:
        bio_embeddings = np.array([user["bio_embedding"] for user in users_data])
        
        tag_embeddings = []
        for user in users_data:
            if hasattr(self.tag_distance, '__name__') and self.tag_distance.__name__ == '_semantic_tag_distance':
                tags_text = " ".join(user["tags"])
                tag_emb = self.embedder.encode(tags_text, convert_to_tensor=False)
                tag_embeddings.append(tag_emb)
            else:
                # for jaccard
                all_tags = list(set(tag for user in users_data for tag in user["tags"]))
                tag_vec = np.zeros(len(all_tags))
                for tag in user["tags"]:
                    if tag in all_tags:
                        tag_vec[all_tags.index(tag)] = 1
                tag_embeddings.append(tag_vec)
        
        tag_embeddings = np.array(tag_embeddings)
        
        bio_embeddings = bio_embeddings / np.linalg.norm(bio_embeddings, axis=1, keepdims=True)
        tag_embeddings = tag_embeddings / np.linalg.norm(tag_embeddings, axis=1, keepdims=True)
        
        combined_embeddings = np.hstack([
            self.coef_a * bio_embeddings,
            self.coef_b * tag_embeddings
        ])
        
        kmeans = KMeans(
            n_clusters=min(self.n_clusters, len(users_data)//2),
            init='k-means++',
            random_state=42
        )
        labels = kmeans.fit_predict(combined_embeddings)
        return labels, kmeans.cluster_centers_

    def compute_pair_score(self, user1: Dict, user2: Dict, history: List[Dict]) -> float:
        bio_sim = cosine_similarity([user1["bio_embedding"]], [user2["bio_embedding"]])[0][0]
        
        tag_dist = self.tag_distance(user1["tags"], user2["tags"])
        
        base_score = self.coef_a * bio_sim + self.coef_b * (1 - tag_dist)

        weeks_passed = self.get_weeks_since_last_meeting(user1["user_id"], user2["user_id"], history)
        penalty = self.penalty ** weeks_passed if weeks_passed != float("inf") else 0

        return base_score * (1 - penalty)

    def get_weeks_since_last_meeting(self, user_id1: str, user_id2: str, history: List[Dict]) -> float:
        for event in history:
            if {event["user_id1"], event["user_id2"]} == {user_id1, user_id2}:
                last_meeting = datetime.strptime(event["timestamp"], "%Y-%m-%d")
                return (datetime.now() - last_meeting).days / 7
        return float("inf")

    def initial_solution(self, clustered_users: DefaultDict[int, List[Dict]]) -> List[List[Optional[Dict]]]:
        solution = []
        leftover_users = []
        
        for cluster_id, users in clustered_users.items():
            random.shuffle(users)
            for i in range(0, len(users)-1, 2):
                solution.append([users[i], users[i+1]])
            if len(users) % 2 != 0:
                leftover_users.append(users[-1])
        
        random.shuffle(leftover_users)
        for i in range(0, len(leftover_users)-1, 2):
            solution.append([leftover_users[i], leftover_users[i+1]])
        if len(leftover_users) % 2 != 0:
            solution.append([leftover_users[-1], None])
            
        return solution

    def energy(self, solution: List[List[Optional[Dict]]], history: List[Dict]) -> float:
        scores = [self.compute_pair_score(p[0], p[1], history) for p in solution if p[1] is not None]
        return -np.mean(scores) if scores else float('inf')

    def get_neighbor(self, solution: List[List[Optional[Dict]]]) -> List[List[Optional[Dict]]]:
        neighbor = deepcopy(solution)
        
        proper_pairs = [p for p in neighbor if p[1] is not None]
        if len(proper_pairs) < 2:
            return neighbor
            
        if random.random() < 0.7:  # intra-cluster swap
            cluster_pairs = [p for p in proper_pairs if p[0]["cluster_id"] == p[1]["cluster_id"]]
            if len(cluster_pairs) >= 2:
                i, j = random.sample(range(len(cluster_pairs)), 2)
                swap_idx = random.randint(0, 1)
                cluster_pairs[i][swap_idx], cluster_pairs[j][swap_idx] = cluster_pairs[j][swap_idx], cluster_pairs[i][swap_idx]
        else:  # inter-cluster move
            i, j = random.sample(range(len(proper_pairs)), 2)
            swap_idx = random.randint(0, 1)
            proper_pairs[i][swap_idx], proper_pairs[j][swap_idx] = proper_pairs[j][swap_idx], proper_pairs[i][swap_idx]
        
        return neighbor

    def match(self, users_data: List[Dict], history: Optional[List[Dict]] = None) -> Tuple[List[Tuple[str, Optional[str], float]], float, float]:
        if history is None:
            history = []
            
        for user in users_data:
            user["bio_embedding"] = self.embedder.encode(user["bio"])
            if hasattr(user["bio_embedding"], "cpu"):
                user["bio_embedding"] = user["bio_embedding"].cpu().numpy()

        cluster_labels, cluster_centers = self.cluster_users(users_data)
        for i, user in enumerate(users_data):
            user["cluster_id"] = int(cluster_labels[i])
            user["cluster_embedding"] = cluster_centers[cluster_labels[i]]
        
        clustered_users = defaultdict(list)
        for user in users_data:
            clustered_users[user["cluster_id"]].append(user)
        
        current_solution = self.initial_solution(clustered_users)
        current_energy = self.energy(current_solution, history)
        best_solution = deepcopy(current_solution)
        best_energy = current_energy
        temp = self.initial_temp
        
        for _ in range(self.n_iterations):
            neighbor = self.get_neighbor(current_solution)
            neighbor_energy = self.energy(neighbor, history)

            if neighbor_energy < current_energy or random.random() < exp((current_energy - neighbor_energy)/temp):
                current_solution = neighbor
                current_energy = neighbor_energy
                if neighbor_energy < best_energy:
                    best_solution = deepcopy(neighbor)
                    best_energy = neighbor_energy
            
            temp *= self.cooling_rate
        
        result = []
        valid_scores = []
        cluster_matches = 0
        
        for pair in best_solution:
            user1, user2 = pair[0], pair[1]
            score = self.compute_pair_score(user1, user2, history) if user2 else 0.0
            result.append((user1["user_id"], user2["user_id"] if user2 else None, score))
            
            if user2:
                valid_scores.append(score)
                if user1["cluster_id"] == user2["cluster_id"]:
                    cluster_matches += 1
        
        avg_score = np.mean(valid_scores) if valid_scores else 0.0
        cluster_homogeneity = cluster_matches / len(valid_scores) if valid_scores else 0.0
        
        return result, avg_score, cluster_homogeneity

In [42]:
matcher = OptimizedClusterAnnealingMatcher(
    n_clusters=5,
    coef_a=0.7,
    coef_b=0.3,
    initial_temp=1000,
    cooling_rate=0.9,
    n_iterations=100,
    penalty_multiplier=0.9,
    embedding_model="all-MiniLM-L6-v2",
    tag_distance_func="semantic"
)

result, avg_score, cluster_homogeneity = matcher.match(users_data, history)
print(f"Average score: {avg_score}")
for user1, user2, score in result:
    print(f"({user1}, {user2})")

/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/rail/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Average score: 0.6503289341926575
(1, 14)
(10, 26)
(3, 2)
(18, 25)
(17, 19)
(23, 11)
(20, 29)
(24, 21)
(7, 4)
(31, 5)
(9, 8)
(6, 27)
(15, 28)
(30, 12)
(13, 22)
(16, None)
